In [ ]:
!pip install boto3
import requests
import json
import boto3

access_key = "###"
secret_key = "######"

sqs_client = boto3.client("sqs",aws_access_key_id = str(access_key),aws_secret_access_key = str(secret_key),region_name='ap-south-1')
queue_url = 'https://sqs.ap-south-1.amazonaws.com/992382564165/stockmarketqueue'

json_list = []

def stock_data():
	url = "https://latest-stock-price.p.rapidapi.com/price"

	querystring = {"Indices":"NIFTY 50"}

	headers = {
		"X-RapidAPI-Key": "####",
		"X-RapidAPI-Host": "latest-stock-price.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers, params=querystring)

	json_list.append(response.json())

	return response.json()



#stock_data()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 9.6 MB/s eta 0:00:00


# SQS insert

In [ ]:
import time

count = 0

while True:
  if count==60:
    break
  response = sqs_client.send_message(
      QueueUrl=queue_url,
      MessageBody=json.dumps(stock_data()))
  count+=1
  time.sleep(1)





# Dynamo DB Upsert

In [ ]:
def upsert_item(table, update_values):
    try:
        response = table.update_item(
            Key={'symbol':update_values['symbol']},
            UpdateExpression='SET #attr1 = :val1, #attr2 = :val2, #attr3 = :val3',
            ExpressionAttributeNames={
                '#attr1': 'lastPrice',
                '#attr2': 'dayHigh',
                '#attr3': 'dayLow'
            },
            ExpressionAttributeValues={
                ':val1': update_values['lastPrice'],
                ':val2': update_values['dayHigh'],
                ':val3': update_values['dayLow']
            },
            ReturnValues='UPDATED_NEW'
        )
        print("Item updated successfully:", response)
    except dynamodb.meta.client.exceptions.ResourceNotFoundException:
        print("Item does not exist, inserting new item...")
        response = table.put_item(Item={
            'symbol': update_values['symbol'],
            'lastPrice': update_values['lastPrice'],
            'dayHigh': update_values['dayHigh'],
            'dayLow': update_values['dayLow']
        })
        print("New item inserted successfully:", response)

# # Define the key and values for the upsert operation
# key = {'key_attribute': 'key_value'}
# update_values = {'value1': 'new_value1', 'value2': 'new_value2'}

def convert_numbers_to_strings(obj):
    if isinstance(obj, (int,float)):
        return str(obj)
    return obj

# Perform the upsert operation
with open('test_data.json') as f:
    message = json.load(f, parse_float=Decimal,object_hook=convert_numbers_to_strings)

for eachgmsg in message[0]:
  upsert_item(dbtable, eachgmsg)

Item updated successfully: {'Attributes': {'dayLow': Decimal('1216.75'), 'dayHigh': Decimal('1268.9'), 'lastPrice': Decimal('0')}, 'ResponseMetadata': {'RequestId': 'FEDTTTSNL8LDMDRALVDQ204I23VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 11 Mar 2024 17:33:10 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '88', 'connection': 'keep-alive', 'x-amzn-requestid': 'FEDTTTSNL8LDMDRALVDQ204I23VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '347806035'}, 'RetryAttempts': 0}}
Item updated successfully: {'Attributes': {'dayLow': Decimal('152.25'), 'dayHigh': Decimal('159.15'), 'lastPrice': Decimal('157.05')}, 'ResponseMetadata': {'RequestId': 'IVA9IDL0NH4RPBNJ89QQN84BJJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 11 Mar 2024 17:33:10 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '92', 'connection': 'keep-alive', 'x-amzn-requestid': 'IVA9IDL0NH4

# AWS Lambda code

In [ ]:
import json
import boto3
from decimal import Decimal
import ast
import datetime


dynamodb = boto3.resource('dynamodb')

dbtable = dynamodb.Table('rapid_api_db')

def send_email(dbtable):
    client = boto3.client("ses")
    response = dbtable.scan()
    data = response['Items']

    sub = 'New stocks data received!'
    text = "New stock data for Nifty has been successfully updated. Below is the latest data from DB. \n \n"
    keys = data[0].keys()


    html = """
          <html>
          <head>
          <style>
          table, th, td {
            border: 1px solid black;
            border-collapse: collapse;
          }
          </style>
          </head>
          <body>
          <table>"""


    html += "<tr>"
    for key in keys:
        html += f"<th>{key}</th>"
    html += "</tr>"


    for row in data:
        html += "<tr>"
        for key in keys:
            html += f"<td>{row[key]}</td>"
        html += "</tr>"


    html += """</table>
                </body>
                </html>"""
    message = {"Subject": {"Data": sub}, "Body": {"Html": {"Data": text + html}}}
    response = client.send_email(Source = "####@gmail.com",
                   Destination = {"ToAddresses": ["####@gmail.com"]}, Message = message)



def upsert_item(table, update_values):
    try:
        response = table.update_item(
            Key={'symbol':update_values['symbol']},
            UpdateExpression='SET #attr1 = :val1, #attr2 = :val2, #attr3 = :val3, #attr4 = :val4',
            ExpressionAttributeNames={
                '#attr1': 'lastPrice',
                '#attr2': 'dayHigh',
                '#attr3': 'dayLow',
                '#attr4': 'UpdateDate'
            },
            ExpressionAttributeValues={
                ':val1': update_values['lastPrice'],
                ':val2': update_values['dayHigh'],
                ':val3': update_values['dayLow'],
                ':val4': str(datetime.datetime.now())
            },
            ReturnValues='UPDATED_NEW'
        )
        print("Item updated successfully:", response)
    except dynamodb.meta.client.exceptions.ResourceNotFoundException:
        print("Item does not exist, inserting new item...")
        response = table.put_item(Item={
            'symbol': update_values['symbol'],
            'lastPrice': update_values['lastPrice'],
            'dayHigh': update_values['dayHigh'],
            'dayLow': update_values['dayLow'],
            'UpdateDate': str(datetime.datetime.now())
        })
        #print("New item inserted successfully:", response)

def convert_numbers_to_strings(obj):
    if isinstance(obj, (int,float)):
        return str(obj)
    return obj

def lambda_handler(event, context):


    records = event['Records']


    list_msg = ast.literal_eval(records[0]['body'].replace("'", "\""))

    for message in list_msg:
        message_str = {key: str(value) for key, value in message.items()}
        upsert_item(dbtable, message_str)

    send_email(dbtable)

# Dynamo get data

In [ ]:
dynamodb = boto3.resource('dynamodb',region_name='ap-south-1',aws_access_key_id = access_key,aws_secret_access_key = secret_key)
dbtable = dynamodb.Table('rapid_api_db')

response = dbtable.scan()
data = response['Items']

print(data[0])

{'dayLow': '271.65', 'UpdateDate': '2024-03-29 07:59:49.646226', 'dayHigh': '279.5', 'symbol': 'POWERGRID', 'lastPrice': '277.35'}


In [ ]:
from tabulate import tabulate

table = tabulate(data, headers='keys', tablefmt='grid')
print(table)

+----------+----------------------------+-----------+------------+-------------+
|   dayLow | UpdateDate                 |   dayHigh | symbol     |   lastPrice |
+==========+============================+===========+============+=============+
|   271.65 | 2024-03-29 07:59:49.646226 |    279.5  | POWERGRID  |      277.35 |
+----------+----------------------------+-----------+------------+-------------+
|   430    | 2024-03-29 07:56:50.467543 |    438.45 | COALINDIA  |      433.95 |
+----------+----------------------------+-----------+------------+-------------+
|   597.6  | 2024-03-29 07:56:50.657615 |    611.25 | BPCL       |      604    |
+----------+----------------------------+-----------+------------+-------------+
|  1618.35 | 2024-03-17 10:45:50.800940 |   1645.8  | INFY       |     1631.45 |
+----------+----------------------------+-----------+------------+-------------+
|  9641.55 | 2024-03-29 07:56:50.837632 |   9853.95 | ULTRACEMCO |     9725    |
+----------+----------------

In [ ]:
from prettytable import PrettyTable
client_ses = boto3.client("ses",region_name='ap-south-1',aws_access_key_id = access_key,aws_secret_access_key = secret_key)
response = dbtable.scan()
data = response['Items']
table = PrettyTable()

# Add column headers based on dictionary keys
table.field_names = data[0].keys()

# Add rows from the list of dictionaries
for emp in data:
    table.add_row(emp.values())

subject = "New stock data received"
body = f"New stock data for Nifty has been successfully updated. Below is the latest data from DB. \n \n {table}"
message = {"Subject": {"Data": subject}, "Body": {"Html": {"Data": body}}}
response = client_ses.send_email(Source = "pradcence16@gmail.com",
               Destination = {"ToAddresses": ["pradcence16@gmail.com"]}, Message = message)

In [ ]:
sub = 'New stocks data received!'
text = "New stock data for Nifty has been successfully updated. Below is the latest data from DB. \n \n"
keys = data[0].keys()


html = """
      <html>
      <head>
      <style>
      table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
      }
      </style>
      </head>
      <body>
      <table>"""


html += "<tr>"
for key in keys:
    html += f"<th>{key}</th>"
html += "</tr>"


for row in data:
    html += "<tr>"
    for key in keys:
        html += f"<td>{row[key]}</td>"
    html += "</tr>"


html += """</table>
            </body>
            </html>"""

message = {"Subject": {"Data": sub}, "Body": {"Html": {"Data": text + html}}}
response = client_ses.send_email(Source = "pradcence16@gmail.com",
               Destination = {"ToAddresses": ["pradcence16@gmail.com"]}, Message = message)